In [1]:
import cv2
import numpy as np
import mediapipe as mp

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

# Initialize the camera
cap = cv2.VideoCapture(0)
canvas = np.zeros((480, 640, 3), dtype=np.uint8)

prev_x, prev_y = None, None
start_x = None
erasing_gesture_active = False

def count_fingers(hand_landmarks):
    fingers = []
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x
    thumb_ip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP].x

    # Thumb
    if thumb_tip < thumb_ip:
        fingers.append(1)
    else:
        fingers.append(0)

    # Other four fingers
    for id in [mp_hands.HandLandmark.INDEX_FINGER_TIP, mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
               mp_hands.HandLandmark.RING_FINGER_TIP, mp_hands.HandLandmark.PINKY_TIP]:
        finger_tip = hand_landmarks.landmark[id].y
        finger_dip = hand_landmarks.landmark[id - 2].y
        if finger_tip < finger_dip:
            fingers.append(1)
        else:
            fingers.append(0)

    return fingers.count(1)

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb_frame)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            h, w, _ = frame.shape
            cx, cy = int(index_finger_tip.x * w), int(index_finger_tip.y * h)

            # Check the number of extended fingers
            fingers_count = count_fingers(hand_landmarks)

            if fingers_count == 5:
                if start_x is None:
                    start_x = cx
                    erasing_gesture_active = True
                elif erasing_gesture_active and cx < start_x - 100:
                    canvas = np.zeros((480, 640, 3), dtype=np.uint8)
                    start_x = None
                    erasing_gesture_active = False
            else:
                start_x = None
                erasing_gesture_active = False

            if not erasing_gesture_active and fingers_count == 1:
                # Check if the index finger is up (using a threshold)
                if hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].y < hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].y:
                    if prev_x is not None and prev_y is not None:
                        cv2.line(canvas, (prev_x, prev_y), (cx, cy), (0, 255, 0), 4)
                    prev_x, prev_y = cx, cy
                else:
                    prev_x, prev_y = None, None
            else:
                prev_x, prev_y = None, None

    frame = cv2.add(frame, canvas)
    cv2.imshow("Hand Drawing", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


D:\python3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
